<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Hello_World.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Function

In [ ]:
!pip install imbalanced-learn
!pip install ipywidgets

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from joblib import Parallel, delayed
from google.colab import files
import joblib
import numpy as np
import pandas as pd


def dataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)


def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)


def save_model(model,model_name):
 joblib.dump(model, model_name)
  

def load_model(model_name):
  return joblib.load(model_name)


def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  return dic



def divide_dataset(dataset,percentage,random_state = 0):
  rows,cols = dataset.shape
  if random_state == 0:
    return dataset.iloc[:int(rows*(percentage)),:]
  else:
    import random
    l_r = random.sample(range(0, rows), int(rows*(percentage)))
    return dataset.iloc[l_r,:]



def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)



def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)

#Starting

In [49]:
dataset = pd.read_csv("/content/drive/MyDrive/Sleep Stage 5 class/EEG_HMC_FeatureExtraction_2023.01.19.csv")
print(dataset.shape)
dataset.dropna(axis = 0,how='any', inplace = True)
dataset = dataset.reset_index(drop = True)
"""df.to_csv("New_EEG_NullValueRemoved_HMC.csv",index = False)
dataset = pd.read_csv("New_EEG_NullValueRemoved_HMC.csv")"""
target = "Sleep Stage"
result = {}
print(dataset.shape)

#Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

dataset = divide_dataset(dataset,0.02)
print(dataset.shape)

(108920, 52)
(108450, 52)
(2169, 52)


###Spliting into X and y

In [43]:
X =  dataset.loc[:,dataset.columns != target]  # removing Sleep Stage
X =  X.loc[:,X.columns != "Subject"]            # removing Status
X =  X.loc[:,X.columns != "Epoch"]             # removing Epoch
y = dataset[target]

#Feature Selection

In [50]:
number_of_feat = 15

###ANOVA with f classifciation

In [51]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd



fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA'] 

lyst = featureScores.nlargest(number_of_feat,'Score_ANOVA')

#lyst.to_csv('Filter_Method_ANOVA_with_f_classif.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "ANOVA"

###Embedded Method

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()

list_of_feat=[]


for i in range(coef.shape[0]):
  if coef[i]!=0:
    list_of_feat.append(dataset.iloc[:0,i+3].name)
    
df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Embedded_Method.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
selection_method = "Embedded"

###Pearson's with f regression

In [ ]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd


fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(dataset.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_pearsons'] 


lyst = featureScores.nlargest(number_of_feat,'Score_pearsons')

#lyst.to_csv('Filter_Method_Pearson’s_with_f_regression.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "Pearson"

###Sequential Feature Selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=number_of_feat)
sfs.fit(X, y)
list_of_feat=[]
list_of_feat=list(sfs.get_feature_names_out(X.columns))

df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Filter_Method_Sequential_feat_Selection_KNN.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]

selection_method = "Sequential"

#Feature list

In [52]:
X=dataset[list_of_feat]
y=dataset[target]
print(X.shape)

(2169, 15)


#Tunning for Scalling and Data Balancing

##Smote : NO
##Scaling : NO

In [67]:
def tune_1():
  X_new = X
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (Standard)

In [68]:
def tune_2():
  X_new = StandardScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (MinMax)

In [69]:
def tune_3():
  X_new = MinMaxScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : Train (Stadard)

In [70]:
def tune_4():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : NO
##Scaling : Train (MinMax)

In [71]:

def tune_5():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : ALL
##Scaling : NO

In [72]:

def tune_6():
  X_new,y_new = Xy_balance(X,y)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)


##Smote : Train
##Scaling : NO

In [73]:

def tune_7():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  return X_train, X_test, y_train, y_test


##Smote : Train
##Scaling : Train (Standard)

In [74]:
def tune_8():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : Train
##Scaling : Train (MixMax)

In [75]:
def tune_9():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : All
##Scaling : Train (Standard)

In [76]:
def tune_10():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : Train (MixMax)

In [77]:
def tune_11():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : All (Standard)

In [78]:
def tune_12():
  X_new,y_new = Xy_balance(X,y)
  X_new = StandardScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : All
##Scaling : ALL (MinMax)

In [79]:
def tune_13():
  X_new,y_new = Xy_balance(X,y)
  X_new = MinMaxScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Tunn Dictionary

In [80]:
tune_dic={
  "Smote : NO    ; Scaling : NO"                  : tune_1(),
  "Smote : NO    ; Scaling : All Data(Standard)"  : tune_2(),
  "Smote : NO    ; Scaling : All Data(MinMax)"    : tune_3(),
  #"Smote : NO    ; Scaling : Train(Standard)"     : tune_4(),
  #"Smote : NO    ; Scaling : Train(MinMax)"       : tune_5(),
  "Smote : ALL   ; Scaling : NO"                  : tune_6(),
  "Smote : Train ; Scaling : NO"                  : tune_7(),
  #"Smote : Train ; Scaling : Train(Standard)"     : tune_8(),
  #"Smote : Train ; Scaling : Train(MinMax)"       : tune_9(),
  #"Smote : All   ; Scaling : Train(Standard)"     : tune_10(),
  #"Smote : All   ; Scaling : Train(MinMax)"       : tune_11(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_12(),
  "Smote : All   ; Scaling : All(MinMax)"        : tune_13()
}



---



# **Training**

##ADABOOST

In [ ]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ada_defult = AdaBoostClassifier(random_state=0)

ada={}
tunning_type = []
accuracy = []
model_name = []
feature_selection_method = []
number_of_features = []



for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  ada_defult.fit(X_train, y_train)
  y_pred = ada_defult.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  tunning_type.append(i)
  accuracy.append(accu)
  model_name.append(ada_defult)
  feature_selection_method.append(selection_method)
  number_of_features.append(number_of_feat)
  
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

ada={
    "model_name"  : model_name,
    "tunning_type" : tunning_type,
    "accurecy"    : accuracy, 
    "feature_selection_method" : feature_selection_method,
    "number_of_features" : number_of_features
}


df = pd.DataFrame.from_dict(ada)
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_ada.csv")



---



---



##Graddient Boosting

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gradBoost_default = GradientBoostingClassifier(random_state=0)

grad={}
tunning_type = []
accuracy = []
model_name = []
feature_selection_method = []
number_of_features = []

for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  gradBoost_default.fit(X_train, y_train)
  y_pred = gradBoost_default.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  tunning_type.append(i)
  accuracy.append(accu)
  model_name.append(gradBoost_default)
  feature_selection_method.append(selection_method)
  number_of_features.append(number_of_feat)
  
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

def tune_csv(model_name,tunning_type,accuracy,feature_selection_method,number_of_features)
grad={
    "model_name"  : model_name,
    "tunning_type" : tunning_type,
    "accurecy"    : accuracy, 
    "feature_selection_method" : feature_selection_method,
    "number_of_features" : number_of_features
}
df = pd.DataFrame.from_dict(grad)
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_grad.csv")

###Histogram-Based Gradient Boosting

In [ ]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier

hisgradBoost_default = HistGradientBoostingClassifier(random_state=0)

hisgrad={}
tunning_type = []
accuracy = []
model_name = []
feature_selection_method = []
number_of_features = []

for i in list(tune_dic.keys()):
  X_train, X_test, y_train, y_test = tune_dic[i]
  
  hisgradBoost_default.fit(X_train, y_train)
  y_pred = hisgradBoost_default.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  tunning_type.append(i)
  accuracy.append(accu)
  model_name.append(hisgradBoost_default)
  feature_selection_method.append(selection_method)
  number_of_features.append(number_of_feat)
  
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

hisgrad={
    "model_name"  : model_name,
    "tunning_type" : tunning_type,
    "accurecy"    : accuracy, 
    "feature_selection_method" : feature_selection_method,
    "number_of_features" : number_of_features
}


df = pd.DataFrame.from_dict(hisgrad)
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_hisgrad.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.71572153065929
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.71572153065929
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.71572153065929
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.2570308898109728
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.3591055786076533
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.7371157553008014
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.6906869525126786
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.2991701244813278
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.25048189104189916
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.3859186365019783
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.7347570254641371
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 9min 56s, sys: 3.52 s, total: 9min 59s
Wall time: 5min 17s




---



---



##Random Forest

In [ ]:
%%time
RanFor={}

from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0,n_jobs=-1)


for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  rf_default.fit(X_train, y_train)
  y_pred=rf_default.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  RanFor[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(RanFor, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_RanFora.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.7256800368833564
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.7291378515444905
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.7277086214845551
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.16629783310281235
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.4782388197325957
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.8205589936086031
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.7095896726602121
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.4367911479944675
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.23901795678198234
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.4014406005884143
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.8186821548138379
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 31min 21s, sys: 1.51 s, total: 31min 22s
Wall time: 16min 6s




---



---



##XGB

In [ ]:
%%time
import xgboost as xgb
li = ["Smote : NO    ; Scaling : Train(Standard)",
      "Smote : NO    ; Scaling : Train(MinMax)",
      "Smote : Train ; Scaling : Train(Standard)",
      "Smote : Train ; Scaling : Train(MinMax)",
      "Smote : All   ; Scaling : Train(Standard)",
      "Smote : All   ; Scaling : Train(MinMax)"]
xgb_deafult = xgb.XGBClassifier(random_state=0,n_jobs=-1)

xgb={}

for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]

  if i in li:
    xgb_deafult.fit(X_train,y_train.values)
    y_pred = xgb_deafult.predict(X_test.values)
  else:
    xgb_deafult.fit(X_train,y_train)
    y_pred = xgb_deafult.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  xgb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(xgb, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_xgb.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.6432918395573998
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6448132780082988
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6432918395573998
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.17671738128169664
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.4284001844167819
------------------

##KNN

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn_default = KNeighborsClassifier(n_jobs = -1)

knn={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  knn_default.fit(X_train, y_train)
  y_pred=knn_default.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  knn[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(knn, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_knn.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.5696173351775011
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6390963577685569
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6217611802674043
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.26924850161364683
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.2653296449976948
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.7336410672618444
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5167358229598894
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.18275703088981096
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.19430354063102365
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.20604646444151364
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.7362280612762504
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 56min 7s, sys: 52.1 s, total: 56min 59s
Wall time: 48min 57s


##NB

In [ ]:
%%time
from sklearn.naive_bayes import GaussianNB

nb_deafult = GaussianNB()

nb={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  nb_deafult.fit(X_train, y_train)
  y_pred = nb_deafult.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  nb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(nb, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_nb.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.3595666205624712
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.31867219917012446
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.31867219917012446
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.10018441678192716
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.11350852927616413
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.2012529167089378
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.15776855693868141
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Smote : Train ; Scaling : Train(Standard)
Accurecy:  0.09990779160903643
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.11811894882434301
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.17566196611545096
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.2111950897839099
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.3556355889215786
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 2.51 s, sys: 191 ms, total: 2.7 s
Wall time: 2.68 s


##Support Vector Machines

####SVM

In [ ]:
"""%%time
from sklearn import svm

svm = svm.SVC(decision_function_shape='ovo')

svm_svc={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  svm.fit(X_train, y_train)
  y_pred = svm.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  svm_svc[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(svm_svc, orient="index")
df.to_csv("svm_svc.csv")"""

'%%time\nfrom sklearn import svm\n\nsvm = svm.SVC(decision_function_shape=\'ovo\')\n\nsvm_svc={}\nfor i in list(tune_dic.keys()):\n\n  X_train, X_test, y_train, y_test = tune_dic[i]\n  svm.fit(X_train, y_train)\n  y_pred = svm.predict(X_test)\n\n\n  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score\n  accu = accuracy_score(y_test, y_pred)\n  print(i)\n  print("Accurecy: ",accu)\n  svm_svc[i] = accu\n  print("------------------------------------------------------------------------------")\n  print("------------------------------------------------------------------------------")\n\ndf = pd.DataFrame.from_dict(svm_svc, orient="index")\ndf.to_csv("svm_svc.csv")'

####Linear

In [ ]:
"""%%time
from sklearn import svm

LinearSVC = svm.LinearSVC()

lin_svc = {}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  LinearSVC.fit(X_train, y_train)
  y_pred = LinearSVC.predict(X_test)



  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  lin_svc[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(lin_svc, orient="index")
df.to_csv("lin_svc.csv")"""

'%%time\nfrom sklearn import svm\n\nLinearSVC = svm.LinearSVC()\n\nlin_svc = {}\nfor i in list(tune_dic.keys()):\n\n  X_train, X_test, y_train, y_test = tune_dic[i]\n  LinearSVC.fit(X_train, y_train)\n  y_pred = LinearSVC.predict(X_test)\n\n\n\n  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score\n  accu = accuracy_score(y_test, y_pred)\n  print(i)\n  print("Accurecy: ",accu)\n  lin_svc[i] = accu\n  print("------------------------------------------------------------------------------")\n  print("------------------------------------------------------------------------------")\n\ndf = pd.DataFrame.from_dict(lin_svc, orient="index")\ndf.to_csv("lin_svc.csv")'

##Stochastic Gradient Descent

In [ ]:
"""%%time
from sklearn.linear_model import SGDClassifier

SGDClassifier = SGDClassifier()

sgdc={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  SGDClassifier.fit(X_train, y_train)
  y_pred = SGDClassifier.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  sgdc[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(sgdc, orient="index")
df.to_csv("sgdc.csv")"""

'%%time\nfrom sklearn.linear_model import SGDClassifier\n\nSGDClassifier = SGDClassifier()\n\nsgdc={}\nfor i in list(tune_dic.keys()):\n\n  X_train, X_test, y_train, y_test = tune_dic[i]\n  SGDClassifier.fit(X_train, y_train)\n  y_pred = SGDClassifier.predict(X_test)\n\n\n  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score\n  accu = accuracy_score(y_test, y_pred)\n  print(i)\n  print("Accurecy: ",accu)\n  sgdc[i] = accu\n  print("------------------------------------------------------------------------------")\n  print("------------------------------------------------------------------------------")\n\ndf = pd.DataFrame.from_dict(sgdc, orient="index")\ndf.to_csv("sgdc.csv")'

##Tree algorithms

###DecisionTreeRegressor

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

dt={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  dtc.fit(X_train, y_train)
  y_pred = dtc.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  dt[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(dt, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_dt.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.5971415398801291
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.602397418165053
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6025818349469801
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.13439372982941447
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.2938681420009221
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.692680328700416
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5833563854310743
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.32706316274781005
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.2053363092218728
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.3004971086537486
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.6938216495891245
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 2min 24s, sys: 144 ms, total: 2min 24s
Wall time: 2min 24s


In [ ]:
!pip install catboost

from catboost import CatBoostClassifier

#cat = CatBoostClassifier(task_type="GPU")

cat=CatBoostClassifier()

catb={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  cat.fit(X_train,y_train)
  y_pred = cat.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  catb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(catb, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_catb.csv")

Streaming output truncated to the last 5000 lines.
24:	learn: 1.0464607	total: 7.93s	remaining: 5m 9s
25:	learn: 1.0409676	total: 8.22s	remaining: 5m 7s
26:	learn: 1.0343618	total: 8.56s	remaining: 5m 8s
27:	learn: 1.0288023	total: 8.9s	remaining: 5m 8s
28:	learn: 1.0244739	total: 9.2s	remaining: 5m 7s
29:	learn: 1.0201091	total: 9.54s	remaining: 5m 8s
30:	learn: 1.0161019	total: 9.8s	remaining: 5m 6s
31:	learn: 1.0121498	total: 10.1s	remaining: 5m 4s
32:	learn: 1.0083197	total: 10.4s	remaining: 5m 3s
33:	learn: 1.0047220	total: 10.6s	remaining: 5m 1s
34:	learn: 1.0005951	total: 10.9s	remaining: 5m
35:	learn: 0.9969384	total: 11.2s	remaining: 4m 58s
36:	learn: 0.9935191	total: 11.4s	remaining: 4m 57s
37:	learn: 0.9900617	total: 11.7s	remaining: 4m 55s
38:	learn: 0.9871317	total: 12s	remaining: 4m 54s
39:	learn: 0.9840074	total: 12.2s	remaining: 4m 53s
40:	learn: 0.9814100	total: 12.5s	remaining: 4m 52s
41:	learn: 0.9786630	total: 12.8s	remaining: 4m 52s
42:	learn: 0.9765379	total: 13s	



---



---

